In [1]:
import os
import pandas as pd
import networkx as nx
import ast
import numpy as np

In [2]:
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
path_out = 'BigData/'
path_in = 'Week_3/data/'
filenames = ['AuthorDataframe.csv', 'PaperDataset.csv', 'PaperAbstractDataset.csv']
frames = []
for filename in filenames:
    frames.append(pd.read_csv(path_in + filename))
df_author, df_paper, df_paper_abs = frames

In [19]:
def string_to_list(inp):
    for char in ['[', ']', ' ', "'"]:
        inp = inp.replace(char, '')
    inp = inp.split(',')
    return inp


def field_from_ID(IDs):
    fields = [[] for _ in IDs]
    # Returns in the order they appear in the dataset
    for i, paper_IDs in enumerate(IDs):
        paper_IDs = ast.literal_eval(paper_IDs)
        paper_IDs = [int(el) if el is not None else None for el in paper_IDs]
        rows = df_author[df_author['AuthorID'].isin(paper_IDs)]['Field']
        rows = [ast.literal_eval(el) for el in rows.values]
        for field_list in rows:
            if field_list is not None:
                fields[i] += list(field_list)
    return list(fields)

In [4]:
field_column = field_from_ID(list(df_paper.authorIds))

In [5]:
df_paper['author_fields'] = field_column
df_paper.to_csv(path_in + 'PaperDataset_af.csv')

In [35]:
df_paper = pd.read_csv(path_in + 'PaperDataset_af.csv')
soc_sci_set = {'Political Science', 'Sociology', 'Economics'}
quant_sci_set = {'Mathematics', 'Physics', 'Computer Science'}

new_data = []
valid = 0
for i, row in df_paper.iterrows():
    # if i == 10:
    #     break
    if type(row['fields']) != float:
        fields_set = set(ast.literal_eval(row['fields']))
        fields = fields_set.intersection(soc_sci_set).union(fields_set.intersection(quant_sci_set))
    else:
        fields = False

    author_set = set(string_to_list(row['author_fields']))
    fields_author = author_set.intersection(quant_sci_set)
    author_len = len(ast.literal_eval(row['authorIds'])) < 10
    date = row['year'] > 2008
    DOI = row['externalId.DOI'] != 'None'

    if date and DOI and (fields or fields_author):
        valid += 1
        new_data.append(row)

In [ ]:
css_papers = pd.DataFrame(new_data)
css_papers.to_csv('BigData/css_papers.csv')

In [32]:
len(new_data), valid

(28376, 9849)

In [29]:
len(field_column)

28376